**1.Target:**

1.  Add Batch normalization and GAP in the architecture

**2.Results**:

1.   Parameters :5150
2.   Best Train Accuracy: 94.06
3.   Best Test Accuracy: 98.07





**3.Analysis:**

Model has 4030 paramter since train accuracy is less than test accuracy hence there is a chances of improvement but ccuracy is decreasing here so we need to add some more optimization technique

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [ ]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [ ]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


In [ ]:
dropout_value = .05
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels= 10,kernel_size= (3, 3), padding=1, bias=False),    #output_image = 28, RF=3
            nn.ReLU(),
            nn.BatchNorm2d(num_features=10),
            nn.Dropout2d(dropout_value)
            )
        #CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10,out_channels= 20,kernel_size= (3, 3), padding=1, bias=False),    #output_image = 28, RF=5
            nn.ReLU(),
            nn.BatchNorm2d(num_features=20),
            nn.Dropout2d(dropout_value)
        )

        #TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2)    #output_image = 14, RF=6
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=20,out_channels= 10,kernel_size= (1, 1), padding=0, bias=False),    #output_image = 14, RF=6
            nn.ReLU(),
            nn.BatchNorm2d(num_features=10),
            nn.Dropout2d(dropout_value)
        )

        #CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=10,out_channels= 10,kernel_size= (3, 3), padding=1, bias=False),    #output_image = 14, RF=10
            nn.ReLU(),
            nn.BatchNorm2d(num_features=10),
            nn.Dropout2d(dropout_value)
        )
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10,out_channels= 20,kernel_size= (3, 3), padding=1, bias=False),    #output_image = 14, RF=14
            nn.ReLU(),
            nn.BatchNorm2d(num_features=20),
            nn.Dropout2d(dropout_value)
        )

        #TRANSITION BLOCK 2
        self.pool2 = nn.MaxPool2d(2, 2)    #output_image = 7, RF=16
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=20,out_channels= 10,kernel_size= (1, 1), padding=0, bias=False),    #output_image = 7, RF=24
            nn.ReLU(),
            nn.BatchNorm2d(num_features=10),
            nn.Dropout2d(dropout_value)
        )

        #CONVOLUTION BLOCK 3
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=7)
        )


    def forward(self, x):
      x = self.convblock1(x)
      x = self.convblock2(x)
      x = self.pool1(x)
      x = self.convblock3(x)
      x = self.convblock4(x)
      x = self.convblock5(x)
      x = self.pool2(x)
      x = self.convblock6(x)
      x = self.gap(x)
      x = x.view(-1, 10)
      return F.log_softmax(x, dim=-1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
         Dropout2d-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 20, 28, 28]           1,800
              ReLU-6           [-1, 20, 28, 28]               0
       BatchNorm2d-7           [-1, 20, 28, 28]              40
         Dropout2d-8           [-1, 20, 28, 28]               0
         MaxPool2d-9           [-1, 20, 14, 14]               0
           Conv2d-10           [-1, 10, 14, 14]             200
             ReLU-11           [-1, 10, 14, 14]               0
      BatchNorm2d-12           [-1, 10, 14, 14]              20

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.778355598449707 Batch_id=468 Accuracy=66.85: 100%|██████████| 469/469 [00:24<00:00, 18.93it/s]



Test set: Average loss: 0.5219, Accuracy: 9310/10000 (93.10%)

EPOCH: 1


Loss=0.4521803855895996 Batch_id=468 Accuracy=85.94: 100%|██████████| 469/469 [00:19<00:00, 23.83it/s]



Test set: Average loss: 0.2767, Accuracy: 9524/10000 (95.24%)

EPOCH: 2


...


Test set: Average loss: 0.0825, Accuracy: 9779/10000 (97.79%)

EPOCH: 13


Loss=0.12771309912204742 Batch_id=468 Accuracy=94.06: 100%|██████████| 469/469 [00:17<00:00, 26.89it/s]



Test set: Average loss: 0.0762, Accuracy: 9807/10000 (98.07%)

EPOCH: 14


Loss=0.19891659915447235 Batch_id=468 Accuracy=94.26: 100%|██████████| 469/469 [00:18<00:00, 25.26it/s]



Test set: Average loss: 0.0764, Accuracy: 9787/10000 (97.87%)



In [ ]:
t = [t_items.item() for t_items in train_losses]
%matplotlib inline
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2,2,figsize=(15,10))
axs[0, 0].plot(t)
axs[0, 0].set_title("Training Loss")
axs[1, 0].plot(train_acc)
axs[1, 0].set_title("Training Accuracy")
axs[0, 1].plot(test_losses)
axs[0, 1].set_title("Test Loss")
axs[1, 1].plot(test_acc)
axs[1, 1].set_title("Test Accuracy")